In [ ]:
pip install pennylane scikit-learn matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 110.7 MB/s eta 0:00:00


In [1]:
pip install plotly==5.22.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [2]:
pip install nbformat>=4.2.0

# local quantum + separable basis Fourier classical surrogate on cancer dataset

# 2 qubit, L = 3

In [ ]:
import math
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from numpy.linalg import lstsq

# ─────────────────────────────────────────────────────────────────────────────
# 0.  Reproducibility
# ─────────────────────────────────────────────────────────────────────────────
np.random.seed(42)

# ─────────────────────────────────────────────────────────────────────────────
# 1.  Load & preprocess
#    PCA → 6 dims → StandardScale → map to [0,2π)
# ─────────────────────────────────────────────────────────────────────────────
data  = load_breast_cancer()
X, y  = data.data, data.target

X_pca = PCA(n_components=6).fit_transform(X)
X_s   = StandardScaler().fit_transform(X_pca)
d     = X_s.shape[1]           # =6
z_all = 2*y - 1                # labels in {–1,+1}

# map each feature into [0,2π)
X_min = X_s.min(axis=0)
X_max = X_s.max(axis=0)
X_2pi = (X_s - X_min) / (X_max - X_min) * 2 * np.pi

# ─────────────────────────────────────────────────────────────────────────────
# Q-node definition (shared architecture)
# ─────────────────────────────────────────────────────────────────────────────
n_qubits = 2
L_sur    = 3
chunk    = math.ceil(d / n_qubits)
dev      = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qmodel(x, W):
    for l in range(L_sur):
        # data-reuploading
        for q in range(n_qubits):
            for k in range(chunk):
                idx = q*chunk + k
                if idx < d:
                    qml.RY(x[idx], wires=q)
                    qml.RZ(x[idx], wires=q)
        # variational layer
        qml.StronglyEntanglingLayers(W[l : l+1], wires=list(range(n_qubits)))
    return qml.expval(qml.PauliZ(0))

def model_batch(X, W):
    return np.array([qmodel(x, W) for x in X])

# ─────────────────────────────────────────────────────────────────────────────
# Separable Fourier basis constructor
# ─────────────────────────────────────────────────────────────────────────────
def fourier_separable_matrix(X, L):
    N, d = X.shape
    cols = 1 + 2*L*d
    A = np.zeros((N, cols))
    A[:, 0] = 1
    j = 1
    for k in range(1, L+1):
        for feat in range(d):
            A[:, j    ] = np.cos(k * X[:, feat])
            A[:, j + 1] = np.sin(k * X[:, feat])
            j += 2
    return A

# ─────────────────────────────────────────────────────────────────────────────
# 2–4.  Loop: for each radius, train local Q-model, then classical surrogate
# ─────────────────────────────────────────────────────────────────────────────
radii = list(np.arange(0.5, 3.5, 0.05))
print(" R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad")
print("-----+---------+----------+---------+----------+------------+---------")

for R in radii:
    # carve patch in PCA-space
    mask  = np.all(np.abs(X_s) < R, axis=1)
    X_loc = X_2pi[mask]
    y_loc = z_all[mask]
    n_loc = len(X_loc)
    if n_loc == 0:
        continue

    # — Train a *local* quantum reuploading model on (X_loc, y_loc)
    W_loc = 0.01 * np.random.randn(L_sur, n_qubits, 3)
    opt   = qml.optimize.AdamOptimizer(0.1)
    # for step in range(1, 61):
    #     W_loc, loss_loc = opt.step_and_cost(
    #         lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
    #         W_loc
    #     )
    #     if loss_loc < 1e-2:
    #         break
        # — Train a local quantum model on (X_loc, y_loc)
    for step in range(1, 61):
        W_loc, loss_loc = opt.step_and_cost(
            lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
            W_loc
        )
        # print intermediate loss every 10 steps
        if step % 10 == 0:
            print(f"  [R={R:.2f}] Q-train step {step:2d}, MSE={loss_loc:.3e}")
        if loss_loc < 1e-2:
            break

    # — Evaluate the trained local Q-model
    z_q     = model_batch(X_loc, W_loc)
    calls_q = n_loc
    mse_q   = mean_squared_error(y_loc, z_q)
    r2_q    = r2_score(y_loc, z_q)

    # — Build separable basis on the same patch & fit classical surrogate
    Φ_loc   = fourier_separable_matrix(X_loc, L_sur)
    α_ad    = lstsq(Φ_loc, z_q, rcond=None)[0]
    z_ad    = Φ_loc @ α_ad
    calls_ad = n_loc
    mse_ad  = mean_squared_error(z_q, z_ad)
    r2_ad   = r2_score(z_q, z_ad)

    # — Print all metrics
    print(f"{R:>4.2f} | {calls_q:7d} | {mse_q:8.3e} | {r2_q:7.3f} |"
          f" {calls_ad:8d} | {mse_ad:10.3e} | {r2_ad:8.3f}")


/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


 R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad
-----+---------+----------+---------+----------+------------+---------
  [R=0.50] Q-train step 10, MSE=8.880e-02
  [R=0.50] Q-train step 20, MSE=7.793e-02
  [R=0.50] Q-train step 30, MSE=6.584e-02
  [R=0.50] Q-train step 40, MSE=6.137e-02
  [R=0.50] Q-train step 50, MSE=5.860e-02
  [R=0.50] Q-train step 60, MSE=5.643e-02
0.50 |      15 | 5.619e-02 |   0.774 |       15 |  1.146e-29 |    1.000
  [R=0.55] Q-train step 10, MSE=3.276e-01
  [R=0.55] Q-train step 20, MSE=3.034e-01
  [R=0.55] Q-train step 30, MSE=3.011e-01
  [R=0.55] Q-train step 40, MSE=2.913e-01
  [R=0.55] Q-train step 50, MSE=2.789e-01
  [R=0.55] Q-train step 60, MSE=2.679e-01
0.55 |      29 | 2.675e-01 |   0.531 |       29 |  2.441e-25 |    1.000
  [R=0.60] Q-train step 10, MSE=2.657e-01
  [R=0.60] Q-train step 20, MSE=2.560e-01
  [R=0.60] Q-train step 30, MSE=2.453e-01
  [R=0.60] Q-train step 40, MSE=2.427e-01
  [R=0.60] Q-train step 50, MSE=2.394e-01
  

In [ ]:
import math
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from numpy.linalg import lstsq

# ─────────────────────────────────────────────────────────────────────────────
# 0.  Reproducibility
# ─────────────────────────────────────────────────────────────────────────────
np.random.seed(42)

# ─────────────────────────────────────────────────────────────────────────────
# 1.  Load & preprocess
#    PCA → 6 dims → StandardScale → map to [0,2π)
# ─────────────────────────────────────────────────────────────────────────────
data  = load_breast_cancer()
X, y  = data.data, data.target

X_pca = PCA(n_components=6).fit_transform(X)
X_s   = StandardScaler().fit_transform(X_pca)
d     = X_s.shape[1]           # =6
z_all = 2*y - 1                # labels in {–1,+1}

# map each feature into [0,2π)
X_min = X_s.min(axis=0)
X_max = X_s.max(axis=0)
X_2pi = (X_s - X_min) / (X_max - X_min) * 2 * np.pi

# ─────────────────────────────────────────────────────────────────────────────
# Q-node definition (shared architecture)
# ─────────────────────────────────────────────────────────────────────────────
n_qubits = 2
L_sur    = 3
chunk    = math.ceil(d / n_qubits)
dev      = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qmodel(x, W):
    for l in range(L_sur):
        # data-reuploading
        for q in range(n_qubits):
            for k in range(chunk):
                idx = q*chunk + k
                if idx < d:
                    qml.RY(x[idx], wires=q)
                    qml.RZ(x[idx], wires=q)
        # variational layer
        qml.StronglyEntanglingLayers(W[l : l+1], wires=list(range(n_qubits)))
    return qml.expval(qml.PauliZ(0))

def model_batch(X, W):
    return np.array([qmodel(x, W) for x in X])

# ─────────────────────────────────────────────────────────────────────────────
# Separable Fourier basis constructor
# ─────────────────────────────────────────────────────────────────────────────
def fourier_separable_matrix(X, L):
    N, d = X.shape
    cols = 1 + 2*L*d
    A = np.zeros((N, cols))
    A[:, 0] = 1
    j = 1
    for k in range(1, L+1):
        for feat in range(d):
            A[:, j    ] = np.cos(k * X[:, feat])
            A[:, j + 1] = np.sin(k * X[:, feat])
            j += 2
    return A

# ─────────────────────────────────────────────────────────────────────────────
# 2–4.  Loop: for each radius, train local Q-model, then classical surrogate
# ─────────────────────────────────────────────────────────────────────────────
radii = list(np.arange(1.95, 3.5, 0.05))
print(" R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad")
print("-----+---------+----------+---------+----------+------------+---------")

for R in radii:
    # carve patch in PCA-space
    mask  = np.all(np.abs(X_s) < R, axis=1)
    X_loc = X_2pi[mask]
    y_loc = z_all[mask]
    n_loc = len(X_loc)
    if n_loc == 0:
        continue

    # — Train a *local* quantum reuploading model on (X_loc, y_loc)
    W_loc = 0.01 * np.random.randn(L_sur, n_qubits, 3)
    opt   = qml.optimize.AdamOptimizer(0.1)
    # for step in range(1, 61):
    #     W_loc, loss_loc = opt.step_and_cost(
    #         lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
    #         W_loc
    #     )
    #     if loss_loc < 1e-2:
    #         break
        # — Train a local quantum model on (X_loc, y_loc)
    for step in range(1, 61):
        W_loc, loss_loc = opt.step_and_cost(
            lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
            W_loc
        )
        # print intermediate loss every 10 steps
        if step % 10 == 0:
            print(f"  [R={R:.2f}] Q-train step {step:2d}, MSE={loss_loc:.3e}")
        if loss_loc < 1e-2:
            break

    # — Evaluate the trained local Q-model
    z_q     = model_batch(X_loc, W_loc)
    calls_q = n_loc
    mse_q   = mean_squared_error(y_loc, z_q)
    r2_q    = r2_score(y_loc, z_q)

    # — Build separable basis on the same patch & fit classical surrogate
    Φ_loc   = fourier_separable_matrix(X_loc, L_sur)
    α_ad    = lstsq(Φ_loc, z_q, rcond=None)[0]
    z_ad    = Φ_loc @ α_ad
    calls_ad = n_loc
    mse_ad  = mean_squared_error(z_q, z_ad)
    r2_ad   = r2_score(z_q, z_ad)

    # — Print all metrics
    print(f"{R:>4.2f} | {calls_q:7d} | {mse_q:8.3e} | {r2_q:7.3f} |"
          f" {calls_ad:8d} | {mse_ad:10.3e} | {r2_ad:8.3f}")


/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


 R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad
-----+---------+----------+---------+----------+------------+---------
  [R=1.95] Q-train step 10, MSE=4.052e-01
  [R=1.95] Q-train step 20, MSE=3.631e-01
  [R=1.95] Q-train step 30, MSE=3.311e-01
  [R=1.95] Q-train step 40, MSE=3.191e-01
  [R=1.95] Q-train step 50, MSE=3.071e-01
  [R=1.95] Q-train step 60, MSE=3.026e-01
1.95 |     463 | 3.023e-01 |   0.634 |      463 |  3.855e-02 |    0.870
  [R=2.00] Q-train step 10, MSE=4.145e-01
  [R=2.00] Q-train step 20, MSE=3.775e-01
  [R=2.00] Q-train step 30, MSE=3.431e-01
  [R=2.00] Q-train step 40, MSE=3.212e-01
  [R=2.00] Q-train step 50, MSE=3.086e-01
  [R=2.00] Q-train step 60, MSE=3.043e-01
2.00 |     471 | 3.040e-01 |   0.636 |      471 |  4.557e-02 |    0.851
  [R=2.05] Q-train step 10, MSE=4.266e-01
  [R=2.05] Q-train step 20, MSE=4.029e-01
  [R=2.05] Q-train step 30, MSE=3.559e-01
  [R=2.05] Q-train step 40, MSE=3.379e-01
  [R=2.05] Q-train step 50, MSE=3.246e-01
  

In [ ]:
import math
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from numpy.linalg import lstsq

# ─────────────────────────────────────────────────────────────────────────────
# 0.  Reproducibility
# ─────────────────────────────────────────────────────────────────────────────
np.random.seed(42)

# ─────────────────────────────────────────────────────────────────────────────
# 1.  Load & preprocess
#    PCA → 6 dims → StandardScale → map to [0,2π)
# ─────────────────────────────────────────────────────────────────────────────
data  = load_breast_cancer()
X, y  = data.data, data.target

X_pca = PCA(n_components=6).fit_transform(X)
X_s   = StandardScaler().fit_transform(X_pca)
d     = X_s.shape[1]           # =6
z_all = 2*y - 1                # labels in {–1,+1}

# map each feature into [0,2π)
X_min = X_s.min(axis=0)
X_max = X_s.max(axis=0)
X_2pi = (X_s - X_min) / (X_max - X_min) * 2 * np.pi

# ─────────────────────────────────────────────────────────────────────────────
# Q-node definition (shared architecture)
# ─────────────────────────────────────────────────────────────────────────────
n_qubits = 2
L_sur    = 3
chunk    = math.ceil(d / n_qubits)
dev      = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qmodel(x, W):
    for l in range(L_sur):
        # data-reuploading
        for q in range(n_qubits):
            for k in range(chunk):
                idx = q*chunk + k
                if idx < d:
                    qml.RY(x[idx], wires=q)
                    qml.RZ(x[idx], wires=q)
        # variational layer
        qml.StronglyEntanglingLayers(W[l : l+1], wires=list(range(n_qubits)))
    return qml.expval(qml.PauliZ(0))

def model_batch(X, W):
    return np.array([qmodel(x, W) for x in X])

# ─────────────────────────────────────────────────────────────────────────────
# Separable Fourier basis constructor
# ─────────────────────────────────────────────────────────────────────────────
def fourier_separable_matrix(X, L):
    N, d = X.shape
    cols = 1 + 2*L*d
    A = np.zeros((N, cols))
    A[:, 0] = 1
    j = 1
    for k in range(1, L+1):
        for feat in range(d):
            A[:, j    ] = np.cos(k * X[:, feat])
            A[:, j + 1] = np.sin(k * X[:, feat])
            j += 2
    return A

# ─────────────────────────────────────────────────────────────────────────────
# 2–4.  Loop: for each radius, train local Q-model, then classical surrogate
# ─────────────────────────────────────────────────────────────────────────────
radii = list(np.arange(2.05, 3.5, 0.05))
print(" R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad")
print("-----+---------+----------+---------+----------+------------+---------")

for R in radii:
    # carve patch in PCA-space
    mask  = np.all(np.abs(X_s) < R, axis=1)
    X_loc = X_2pi[mask]
    y_loc = z_all[mask]
    n_loc = len(X_loc)
    if n_loc == 0:
        continue

    # — Train a *local* quantum reuploading model on (X_loc, y_loc)
    W_loc = 0.01 * np.random.randn(L_sur, n_qubits, 3)
    opt   = qml.optimize.AdamOptimizer(0.1)
    # for step in range(1, 61):
    #     W_loc, loss_loc = opt.step_and_cost(
    #         lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
    #         W_loc
    #     )
    #     if loss_loc < 1e-2:
    #         break
        # — Train a local quantum model on (X_loc, y_loc)
    for step in range(1, 61):
        W_loc, loss_loc = opt.step_and_cost(
            lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
            W_loc
        )
        # print intermediate loss every 10 steps
        if step % 10 == 0:
            print(f"  [R={R:.2f}] Q-train step {step:2d}, MSE={loss_loc:.3e}")
        if loss_loc < 1e-2:
            break

    # — Evaluate the trained local Q-model
    z_q     = model_batch(X_loc, W_loc)
    calls_q = n_loc
    mse_q   = mean_squared_error(y_loc, z_q)
    r2_q    = r2_score(y_loc, z_q)

    # — Build separable basis on the same patch & fit classical surrogate
    Φ_loc   = fourier_separable_matrix(X_loc, L_sur)
    α_ad    = lstsq(Φ_loc, z_q, rcond=None)[0]
    z_ad    = Φ_loc @ α_ad
    calls_ad = n_loc
    mse_ad  = mean_squared_error(z_q, z_ad)
    r2_ad   = r2_score(z_q, z_ad)

    # — Print all metrics
    print(f"{R:>4.2f} | {calls_q:7d} | {mse_q:8.3e} | {r2_q:7.3f} |"
          f" {calls_ad:8d} | {mse_ad:10.3e} | {r2_ad:8.3f}")


/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


 R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad
-----+---------+----------+---------+----------+------------+---------
  [R=2.05] Q-train step 10, MSE=4.265e-01
  [R=2.05] Q-train step 20, MSE=3.784e-01
  [R=2.05] Q-train step 30, MSE=3.464e-01
  [R=2.05] Q-train step 40, MSE=3.300e-01
  [R=2.05] Q-train step 50, MSE=3.181e-01
  [R=2.05] Q-train step 60, MSE=3.127e-01
2.05 |     477 | 3.125e-01 |   0.628 |      477 |  4.507e-02 |    0.852
  [R=2.10] Q-train step 10, MSE=4.429e-01
  [R=2.10] Q-train step 20, MSE=4.154e-01
  [R=2.10] Q-train step 30, MSE=3.638e-01
  [R=2.10] Q-train step 40, MSE=3.549e-01
  [R=2.10] Q-train step 50, MSE=3.383e-01
  [R=2.10] Q-train step 60, MSE=3.272e-01
2.10 |     482 | 3.263e-01 |   0.617 |      482 |  3.552e-02 |    0.875
  [R=2.15] Q-train step 10, MSE=4.579e-01
  [R=2.15] Q-train step 20, MSE=4.232e-01
  [R=2.15] Q-train step 30, MSE=3.886e-01
  [R=2.15] Q-train step 40, MSE=3.493e-01
  [R=2.15] Q-train step 50, MSE=3.370e-01
  

In [ ]:
import math
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from numpy.linalg import lstsq

# ─────────────────────────────────────────────────────────────────────────────
# 0.  Reproducibility
# ─────────────────────────────────────────────────────────────────────────────
np.random.seed(42)

# ─────────────────────────────────────────────────────────────────────────────
# 1.  Load & preprocess
#    PCA → 6 dims → StandardScale → map to [0,2π)
# ─────────────────────────────────────────────────────────────────────────────
data  = load_breast_cancer()
X, y  = data.data, data.target

X_pca = PCA(n_components=6).fit_transform(X)
X_s   = StandardScaler().fit_transform(X_pca)
d     = X_s.shape[1]           # =6
z_all = 2*y - 1                # labels in {–1,+1}

# map each feature into [0,2π)
X_min = X_s.min(axis=0)
X_max = X_s.max(axis=0)
X_2pi = (X_s - X_min) / (X_max - X_min) * 2 * np.pi

# ─────────────────────────────────────────────────────────────────────────────
# Q-node definition (shared architecture)
# ─────────────────────────────────────────────────────────────────────────────
n_qubits = 2
L_sur    = 3
chunk    = math.ceil(d / n_qubits)
dev      = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qmodel(x, W):
    for l in range(L_sur):
        # data-reuploading
        for q in range(n_qubits):
            for k in range(chunk):
                idx = q*chunk + k
                if idx < d:
                    qml.RY(x[idx], wires=q)
                    qml.RZ(x[idx], wires=q)
        # variational layer
        qml.StronglyEntanglingLayers(W[l : l+1], wires=list(range(n_qubits)))
    return qml.expval(qml.PauliZ(0))

def model_batch(X, W):
    return np.array([qmodel(x, W) for x in X])

# ─────────────────────────────────────────────────────────────────────────────
# Separable Fourier basis constructor
# ─────────────────────────────────────────────────────────────────────────────
def fourier_separable_matrix(X, L):
    N, d = X.shape
    cols = 1 + 2*L*d
    A = np.zeros((N, cols))
    A[:, 0] = 1
    j = 1
    for k in range(1, L+1):
        for feat in range(d):
            A[:, j    ] = np.cos(k * X[:, feat])
            A[:, j + 1] = np.sin(k * X[:, feat])
            j += 2
    return A

# ─────────────────────────────────────────────────────────────────────────────
# 2–4.  Loop: for each radius, train local Q-model, then classical surrogate
# ─────────────────────────────────────────────────────────────────────────────
radii = list(np.arange(2.5, 3.5, 0.05))
print(" R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad")
print("-----+---------+----------+---------+----------+------------+---------")

for R in radii:
    # carve patch in PCA-space
    mask  = np.all(np.abs(X_s) < R, axis=1)
    X_loc = X_2pi[mask]
    y_loc = z_all[mask]
    n_loc = len(X_loc)
    if n_loc == 0:
        continue

    # — Train a *local* quantum reuploading model on (X_loc, y_loc)
    W_loc = 0.01 * np.random.randn(L_sur, n_qubits, 3)
    opt   = qml.optimize.AdamOptimizer(0.1)
    # for step in range(1, 61):
    #     W_loc, loss_loc = opt.step_and_cost(
    #         lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
    #         W_loc
    #     )
    #     if loss_loc < 1e-2:
    #         break
        # — Train a local quantum model on (X_loc, y_loc)
    for step in range(1, 61):
        W_loc, loss_loc = opt.step_and_cost(
            lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
            W_loc
        )
        # print intermediate loss every 10 steps
        if step % 10 == 0:
            print(f"  [R={R:.2f}] Q-train step {step:2d}, MSE={loss_loc:.3e}")
        if loss_loc < 1e-2:
            break

    # — Evaluate the trained local Q-model
    z_q     = model_batch(X_loc, W_loc)
    calls_q = n_loc
    mse_q   = mean_squared_error(y_loc, z_q)
    r2_q    = r2_score(y_loc, z_q)

    # — Build separable basis on the same patch & fit classical surrogate
    Φ_loc   = fourier_separable_matrix(X_loc, L_sur)
    α_ad    = lstsq(Φ_loc, z_q, rcond=None)[0]
    z_ad    = Φ_loc @ α_ad
    calls_ad = n_loc
    mse_ad  = mean_squared_error(z_q, z_ad)
    r2_ad   = r2_score(z_q, z_ad)

    # — Print all metrics
    print(f"{R:>4.2f} | {calls_q:7d} | {mse_q:8.3e} | {r2_q:7.3f} |"
          f" {calls_ad:8d} | {mse_ad:10.3e} | {r2_ad:8.3f}")


/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


 R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad
-----+---------+----------+---------+----------+------------+---------
  [R=2.50] Q-train step 10, MSE=4.832e-01
  [R=2.50] Q-train step 20, MSE=4.442e-01
  [R=2.50] Q-train step 30, MSE=3.912e-01
  [R=2.50] Q-train step 40, MSE=3.855e-01
  [R=2.50] Q-train step 50, MSE=3.722e-01
  [R=2.50] Q-train step 60, MSE=3.624e-01
2.50 |     513 | 3.612e-01 |   0.587 |      513 |  4.152e-02 |    0.851
  [R=2.55] Q-train step 10, MSE=4.797e-01
  [R=2.55] Q-train step 20, MSE=4.104e-01
  [R=2.55] Q-train step 30, MSE=3.923e-01
  [R=2.55] Q-train step 40, MSE=3.755e-01
  [R=2.55] Q-train step 50, MSE=3.670e-01
  [R=2.55] Q-train step 60, MSE=3.587e-01
2.55 |     517 | 3.577e-01 |   0.595 |      517 |  4.433e-02 |    0.843
  [R=2.60] Q-train step 10, MSE=5.033e-01
  [R=2.60] Q-train step 20, MSE=4.523e-01
  [R=2.60] Q-train step 30, MSE=4.178e-01
  [R=2.60] Q-train step 40, MSE=3.947e-01
  [R=2.60] Q-train step 50, MSE=3.858e-01
  

In [ ]:
import math
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from numpy.linalg import lstsq

# ─────────────────────────────────────────────────────────────────────────────
# 0.  Reproducibility
# ─────────────────────────────────────────────────────────────────────────────
np.random.seed(42)

# ─────────────────────────────────────────────────────────────────────────────
# 1.  Load & preprocess
#    PCA → 6 dims → StandardScale → map to [0,2π)
# ─────────────────────────────────────────────────────────────────────────────
data  = load_breast_cancer()
X, y  = data.data, data.target

X_pca = PCA(n_components=6).fit_transform(X)
X_s   = StandardScaler().fit_transform(X_pca)
d     = X_s.shape[1]           # =6
z_all = 2*y - 1                # labels in {–1,+1}

# map each feature into [0,2π)
X_min = X_s.min(axis=0)
X_max = X_s.max(axis=0)
X_2pi = (X_s - X_min) / (X_max - X_min) * 2 * np.pi

# ─────────────────────────────────────────────────────────────────────────────
# Q-node definition (shared architecture)
# ─────────────────────────────────────────────────────────────────────────────
n_qubits = 2
L_sur    = 3
chunk    = math.ceil(d / n_qubits)
dev      = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qmodel(x, W):
    for l in range(L_sur):
        # data-reuploading
        for q in range(n_qubits):
            for k in range(chunk):
                idx = q*chunk + k
                if idx < d:
                    qml.RY(x[idx], wires=q)
                    qml.RZ(x[idx], wires=q)
        # variational layer
        qml.StronglyEntanglingLayers(W[l : l+1], wires=list(range(n_qubits)))
    return qml.expval(qml.PauliZ(0))

def model_batch(X, W):
    return np.array([qmodel(x, W) for x in X])

# ─────────────────────────────────────────────────────────────────────────────
# Separable Fourier basis constructor
# ─────────────────────────────────────────────────────────────────────────────
def fourier_separable_matrix(X, L):
    N, d = X.shape
    cols = 1 + 2*L*d
    A = np.zeros((N, cols))
    A[:, 0] = 1
    j = 1
    for k in range(1, L+1):
        for feat in range(d):
            A[:, j    ] = np.cos(k * X[:, feat])
            A[:, j + 1] = np.sin(k * X[:, feat])
            j += 2
    return A

# ─────────────────────────────────────────────────────────────────────────────
# 2–4.  Loop: for each radius, train local Q-model, then classical surrogate
# ─────────────────────────────────────────────────────────────────────────────
radii = list(np.arange(2.65, 3.5, 0.05))
print(" R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad")
print("-----+---------+----------+---------+----------+------------+---------")

for R in radii:
    # carve patch in PCA-space
    mask  = np.all(np.abs(X_s) < R, axis=1)
    X_loc = X_2pi[mask]
    y_loc = z_all[mask]
    n_loc = len(X_loc)
    if n_loc == 0:
        continue

    # — Train a *local* quantum reuploading model on (X_loc, y_loc)
    W_loc = 0.01 * np.random.randn(L_sur, n_qubits, 3)
    opt   = qml.optimize.AdamOptimizer(0.1)
    # for step in range(1, 61):
    #     W_loc, loss_loc = opt.step_and_cost(
    #         lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
    #         W_loc
    #     )
    #     if loss_loc < 1e-2:
    #         break
        # — Train a local quantum model on (X_loc, y_loc)
    for step in range(1, 61):
        W_loc, loss_loc = opt.step_and_cost(
            lambda w: np.mean((model_batch(X_loc, w) - y_loc)**2),
            W_loc
        )
        # print intermediate loss every 10 steps
        if step % 10 == 0:
            print(f"  [R={R:.2f}] Q-train step {step:2d}, MSE={loss_loc:.3e}")
        if loss_loc < 1e-2:
            break

    # — Evaluate the trained local Q-model
    z_q     = model_batch(X_loc, W_loc)
    calls_q = n_loc
    mse_q   = mean_squared_error(y_loc, z_q)
    r2_q    = r2_score(y_loc, z_q)

    # — Build separable basis on the same patch & fit classical surrogate
    Φ_loc   = fourier_separable_matrix(X_loc, L_sur)
    α_ad    = lstsq(Φ_loc, z_q, rcond=None)[0]
    z_ad    = Φ_loc @ α_ad
    calls_ad = n_loc
    mse_ad  = mean_squared_error(z_q, z_ad)
    r2_ad   = r2_score(z_q, z_ad)

    # — Print all metrics
    print(f"{R:>4.2f} | {calls_q:7d} | {mse_q:8.3e} | {r2_q:7.3f} |"
          f" {calls_ad:8d} | {mse_ad:10.3e} | {r2_ad:8.3f}")


/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


 R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad
-----+---------+----------+---------+----------+------------+---------
  [R=2.65] Q-train step 10, MSE=5.082e-01
  [R=2.65] Q-train step 20, MSE=4.376e-01
  [R=2.65] Q-train step 30, MSE=4.113e-01
  [R=2.65] Q-train step 40, MSE=3.913e-01
  [R=2.65] Q-train step 50, MSE=3.810e-01
  [R=2.65] Q-train step 60, MSE=3.686e-01
2.65 |     526 | 3.677e-01 |   0.587 |      526 |  4.689e-02 |    0.838
  [R=2.70] Q-train step 10, MSE=5.018e-01
  [R=2.70] Q-train step 20, MSE=4.499e-01
  [R=2.70] Q-train step 30, MSE=4.136e-01
  [R=2.70] Q-train step 40, MSE=3.976e-01
  [R=2.70] Q-train step 50, MSE=3.880e-01
  [R=2.70] Q-train step 60, MSE=3.731e-01
2.70 |     528 | 3.718e-01 |   0.584 |      528 |  4.588e-02 |    0.842
  [R=2.75] Q-train step 10, MSE=5.238e-01
  [R=2.75] Q-train step 20, MSE=4.773e-01
  [R=2.75] Q-train step 30, MSE=4.351e-01
  [R=2.75] Q-train step 40, MSE=4.140e-01
  [R=2.75] Q-train step 50, MSE=4.053e-01
  

# Plots

In [3]:
import re
import pandas as pd
import plotly.express as px

# ------------------------------------------------------------------
# 1.  Paste the console dump from your two simulations verbatim here
# ------------------------------------------------------------------
raw_data = r"""
 R   | calls_q |  MSE_q   |  R²_q  | calls_ad |   MSE_ad   |   R²_ad
-----+---------+----------+---------+----------+------------+---------
  [R=0.50] Q-train step 10, MSE=8.880e-02
  [R=0.50] Q-train step 20, MSE=7.793e-02
  [R=0.50] Q-train step 30, MSE=6.584e-02
  [R=0.50] Q-train step 40, MSE=6.137e-02
  [R=0.50] Q-train step 50, MSE=5.860e-02
  [R=0.50] Q-train step 60, MSE=5.643e-02
0.50 |      15 | 5.619e-02 |   0.774 |       15 |  1.146e-29 |    1.000
  [R=0.55] Q-train step 10, MSE=3.276e-01
  [R=0.55] Q-train step 20, MSE=3.034e-01
  [R=0.55] Q-train step 30, MSE=3.011e-01
  [R=0.55] Q-train step 40, MSE=2.913e-01
  [R=0.55] Q-train step 50, MSE=2.789e-01
  [R=0.55] Q-train step 60, MSE=2.679e-01
0.55 |      29 | 2.675e-01 |   0.531 |       29 |  2.441e-25 |    1.000
  [R=0.60] Q-train step 10, MSE=2.657e-01
  [R=0.60] Q-train step 20, MSE=2.560e-01
  [R=0.60] Q-train step 30, MSE=2.453e-01
  [R=0.60] Q-train step 40, MSE=2.427e-01
  [R=0.60] Q-train step 50, MSE=2.394e-01
  [R=0.60] Q-train step 60, MSE=2.344e-01
0.60 |      40 | 2.340e-01 |   0.465 |       40 |  3.785e-04 |    0.996
  [R=0.65] Q-train step 10, MSE=2.181e-01
  [R=0.65] Q-train step 20, MSE=1.910e-01
  [R=0.65] Q-train step 30, MSE=1.876e-01
  [R=0.65] Q-train step 40, MSE=1.808e-01
  [R=0.65] Q-train step 50, MSE=1.771e-01
  [R=0.65] Q-train step 60, MSE=1.743e-01
0.65 |      63 | 1.740e-01 |   0.495 |       63 |  2.268e-03 |    0.975
  [R=0.70] Q-train step 10, MSE=1.882e-01
  [R=0.70] Q-train step 20, MSE=1.608e-01
  [R=0.70] Q-train step 30, MSE=1.574e-01
  [R=0.70] Q-train step 40, MSE=1.523e-01
  [R=0.70] Q-train step 50, MSE=1.486e-01
  [R=0.70] Q-train step 60, MSE=1.468e-01
0.70 |      78 | 1.466e-01 |   0.551 |       78 |  3.352e-03 |    0.969
  [R=0.75] Q-train step 10, MSE=1.612e-01
  [R=0.75] Q-train step 20, MSE=1.487e-01
  [R=0.75] Q-train step 30, MSE=1.383e-01
  [R=0.75] Q-train step 40, MSE=1.273e-01
  [R=0.75] Q-train step 50, MSE=1.251e-01
  [R=0.75] Q-train step 60, MSE=1.218e-01
0.75 |     102 | 1.216e-01 |   0.622 |      102 |  3.192e-03 |    0.975
  [R=0.80] Q-train step 10, MSE=1.670e-01
  [R=0.80] Q-train step 20, MSE=1.437e-01
  [R=0.80] Q-train step 30, MSE=1.331e-01
  [R=0.80] Q-train step 40, MSE=1.253e-01
  [R=0.80] Q-train step 50, MSE=1.238e-01
  [R=0.80] Q-train step 60, MSE=1.217e-01
0.80 |     122 | 1.217e-01 |   0.596 |      122 |  3.187e-03 |    0.973
  [R=0.85] Q-train step 10, MSE=1.606e-01
  [R=0.85] Q-train step 20, MSE=1.399e-01
  [R=0.85] Q-train step 30, MSE=1.270e-01
  [R=0.85] Q-train step 40, MSE=1.188e-01
  [R=0.85] Q-train step 50, MSE=1.157e-01
  [R=0.85] Q-train step 60, MSE=1.117e-01
0.85 |     147 | 1.113e-01 |   0.598 |      147 |  2.789e-03 |    0.975
  [R=0.90] Q-train step 10, MSE=1.708e-01
  [R=0.90] Q-train step 20, MSE=1.632e-01
  [R=0.90] Q-train step 30, MSE=1.459e-01
  [R=0.90] Q-train step 40, MSE=1.404e-01
  [R=0.90] Q-train step 50, MSE=1.337e-01
  [R=0.90] Q-train step 60, MSE=1.297e-01
0.90 |     169 | 1.294e-01 |   0.660 |      169 |  4.079e-03 |    0.976
  [R=0.95] Q-train step 10, MSE=1.693e-01
  [R=0.95] Q-train step 20, MSE=1.534e-01
  [R=0.95] Q-train step 30, MSE=1.372e-01
  [R=0.95] Q-train step 40, MSE=1.286e-01
  [R=0.95] Q-train step 50, MSE=1.233e-01
  [R=0.95] Q-train step 60, MSE=1.204e-01
0.95 |     195 | 1.202e-01 |   0.674 |      195 |  3.774e-03 |    0.977
  [R=1.00] Q-train step 10, MSE=1.984e-01
  [R=1.00] Q-train step 20, MSE=1.800e-01
  [R=1.00] Q-train step 30, MSE=1.646e-01
  [R=1.00] Q-train step 40, MSE=1.571e-01
  [R=1.00] Q-train step 50, MSE=1.506e-01
  [R=1.00] Q-train step 60, MSE=1.469e-01
1.00 |     217 | 1.467e-01 |   0.652 |      217 |  6.080e-03 |    0.967
  [R=1.05] Q-train step 10, MSE=2.210e-01
  [R=1.05] Q-train step 20, MSE=2.008e-01
  [R=1.05] Q-train step 30, MSE=1.879e-01
  [R=1.05] Q-train step 40, MSE=1.796e-01
  [R=1.05] Q-train step 50, MSE=1.696e-01
  [R=1.05] Q-train step 60, MSE=1.624e-01
1.05 |     236 | 1.619e-01 |   0.613 |      236 |  7.904e-03 |    0.953
  [R=1.10] Q-train step 10, MSE=2.302e-01
  [R=1.10] Q-train step 20, MSE=2.065e-01
  [R=1.10] Q-train step 30, MSE=1.948e-01
  [R=1.10] Q-train step 40, MSE=1.887e-01
  [R=1.10] Q-train step 50, MSE=1.803e-01
  [R=1.10] Q-train step 60, MSE=1.748e-01
1.10 |     254 | 1.744e-01 |   0.624 |      254 |  1.170e-02 |    0.939
  [R=1.15] Q-train step 10, MSE=2.366e-01
  [R=1.15] Q-train step 20, MSE=2.073e-01
  [R=1.15] Q-train step 30, MSE=1.953e-01
  [R=1.15] Q-train step 40, MSE=1.878e-01
  [R=1.15] Q-train step 50, MSE=1.784e-01
  [R=1.15] Q-train step 60, MSE=1.731e-01
1.15 |     272 | 1.727e-01 |   0.641 |      272 |  1.147e-02 |    0.942
  [R=1.20] Q-train step 10, MSE=2.478e-01
  [R=1.20] Q-train step 20, MSE=2.178e-01
  [R=1.20] Q-train step 30, MSE=2.048e-01
  [R=1.20] Q-train step 40, MSE=1.955e-01
  [R=1.20] Q-train step 50, MSE=1.852e-01
  [R=1.20] Q-train step 60, MSE=1.805e-01
1.20 |     291 | 1.802e-01 |   0.655 |      291 |  8.595e-03 |    0.959
  [R=1.25] Q-train step 10, MSE=2.606e-01
  [R=1.25] Q-train step 20, MSE=2.326e-01
  [R=1.25] Q-train step 30, MSE=2.232e-01
  [R=1.25] Q-train step 40, MSE=2.125e-01
  [R=1.25] Q-train step 50, MSE=2.013e-01
  [R=1.25] Q-train step 60, MSE=1.963e-01
1.25 |     308 | 1.960e-01 |   0.666 |      308 |  1.106e-02 |    0.954
  [R=1.30] Q-train step 10, MSE=2.808e-01
  [R=1.30] Q-train step 20, MSE=2.517e-01
  [R=1.30] Q-train step 30, MSE=2.439e-01
  [R=1.30] Q-train step 40, MSE=2.338e-01
  [R=1.30] Q-train step 50, MSE=2.213e-01
  [R=1.30] Q-train step 60, MSE=2.136e-01
1.30 |     331 | 2.134e-01 |   0.677 |      331 |  1.494e-02 |    0.944
  [R=1.35] Q-train step 10, MSE=2.909e-01
  [R=1.35] Q-train step 20, MSE=2.614e-01
  [R=1.35] Q-train step 30, MSE=2.506e-01
  [R=1.35] Q-train step 40, MSE=2.376e-01
  [R=1.35] Q-train step 50, MSE=2.225e-01
  [R=1.35] Q-train step 60, MSE=2.149e-01
1.35 |     344 | 2.143e-01 |   0.683 |      344 |  1.328e-02 |    0.951
  [R=1.40] Q-train step 10, MSE=3.005e-01
  [R=1.40] Q-train step 20, MSE=2.652e-01
  [R=1.40] Q-train step 30, MSE=2.557e-01
  [R=1.40] Q-train step 40, MSE=2.436e-01
  [R=1.40] Q-train step 50, MSE=2.310e-01
  [R=1.40] Q-train step 60, MSE=2.243e-01
1.40 |     356 | 2.239e-01 |   0.673 |      356 |  1.501e-02 |    0.945
  [R=1.45] Q-train step 10, MSE=3.047e-01
  [R=1.45] Q-train step 20, MSE=2.737e-01
  [R=1.45] Q-train step 30, MSE=2.601e-01
  [R=1.45] Q-train step 40, MSE=2.418e-01
  [R=1.45] Q-train step 50, MSE=2.283e-01
  [R=1.45] Q-train step 60, MSE=2.239e-01
1.45 |     367 | 2.239e-01 |   0.675 |      367 |  1.724e-02 |    0.940
  [R=1.50] Q-train step 10, MSE=3.275e-01
  [R=1.50] Q-train step 20, MSE=2.912e-01
  [R=1.50] Q-train step 30, MSE=2.759e-01
  [R=1.50] Q-train step 40, MSE=2.557e-01
  [R=1.50] Q-train step 50, MSE=2.372e-01
  [R=1.50] Q-train step 60, MSE=2.350e-01
1.50 |     382 | 2.349e-01 |   0.666 |      382 |  1.784e-02 |    0.936
  [R=1.55] Q-train step 10, MSE=3.438e-01
  [R=1.55] Q-train step 20, MSE=3.051e-01
  [R=1.55] Q-train step 30, MSE=2.926e-01
  [R=1.55] Q-train step 40, MSE=2.774e-01
  [R=1.55] Q-train step 50, MSE=2.625e-01
  [R=1.55] Q-train step 60, MSE=2.468e-01
1.55 |     394 | 2.468e-01 |   0.663 |      394 |  1.808e-02 |    0.935
  [R=1.60] Q-train step 10, MSE=3.534e-01
  [R=1.60] Q-train step 20, MSE=3.150e-01
  [R=1.60] Q-train step 30, MSE=3.027e-01
  [R=1.60] Q-train step 40, MSE=2.892e-01
  [R=1.60] Q-train step 50, MSE=2.792e-01
  [R=1.60] Q-train step 60, MSE=2.570e-01
1.60 |     408 | 2.570e-01 |   0.662 |      408 |  2.163e-02 |    0.923
  [R=1.65] Q-train step 10, MSE=3.514e-01
  [R=1.65] Q-train step 20, MSE=3.235e-01
  [R=1.65] Q-train step 30, MSE=3.067e-01
  [R=1.65] Q-train step 40, MSE=2.917e-01
  [R=1.65] Q-train step 50, MSE=2.809e-01
  [R=1.65] Q-train step 60, MSE=2.611e-01
1.65 |     418 | 2.609e-01 |   0.658 |      418 |  2.199e-02 |    0.923
  [R=1.70] Q-train step 10, MSE=3.632e-01
  [R=1.70] Q-train step 20, MSE=3.350e-01
  [R=1.70] Q-train step 30, MSE=3.136e-01
  [R=1.70] Q-train step 40, MSE=3.033e-01
  [R=1.70] Q-train step 50, MSE=2.955e-01
  [R=1.70] Q-train step 60, MSE=2.729e-01
1.70 |     428 | 2.708e-01 |   0.650 |      428 |  2.733e-02 |    0.905
  [R=1.75] Q-train step 10, MSE=3.799e-01
  [R=1.75] Q-train step 20, MSE=3.359e-01
  [R=1.75] Q-train step 30, MSE=3.210e-01
  [R=1.75] Q-train step 40, MSE=3.067e-01
  [R=1.75] Q-train step 50, MSE=2.826e-01
  [R=1.75] Q-train step 60, MSE=2.765e-01
1.75 |     437 | 2.759e-01 |   0.648 |      437 |  2.991e-02 |    0.893
  [R=1.80] Q-train step 10, MSE=3.848e-01
  [R=1.80] Q-train step 20, MSE=3.440e-01
  [R=1.80] Q-train step 30, MSE=3.187e-01
  [R=1.80] Q-train step 40, MSE=2.907e-01
  [R=1.80] Q-train step 50, MSE=2.827e-01
  [R=1.80] Q-train step 60, MSE=2.823e-01
1.80 |     443 | 2.819e-01 |   0.643 |      443 |  2.597e-02 |    0.906
  [R=1.85] Q-train step 10, MSE=4.047e-01
  [R=1.85] Q-train step 20, MSE=3.873e-01
  [R=1.85] Q-train step 30, MSE=3.705e-01
  [R=1.85] Q-train step 40, MSE=3.489e-01
  [R=1.85] Q-train step 50, MSE=3.279e-01
  [R=1.85] Q-train step 60, MSE=3.135e-01
1.85 |     451 | 3.114e-01 |   0.615 |      451 |  4.357e-02 |    0.852
  [R=1.90] Q-train step 10, MSE=3.925e-01
  [R=1.90] Q-train step 20, MSE=3.400e-01
  [R=1.90] Q-train step 30, MSE=3.123e-01
  [R=1.90] Q-train step 40, MSE=3.019e-01
  [R=1.90] Q-train step 50, MSE=2.935e-01
  [R=1.90] Q-train step 60, MSE=2.910e-01
1.90 |     456 | 2.908e-01 |   0.645 |      456 |  4.354e-02 |    0.861
  [R=1.95] Q-train step 10, MSE=4.052e-01
  [R=1.95] Q-train step 20, MSE=3.631e-01
  [R=1.95] Q-train step 30, MSE=3.311e-01
  [R=1.95] Q-train step 40, MSE=3.191e-01
  [R=1.95] Q-train step 50, MSE=3.071e-01
  [R=1.95] Q-train step 60, MSE=3.026e-01
1.95 |     463 | 3.023e-01 |   0.634 |      463 |  3.855e-02 |    0.870
  [R=2.00] Q-train step 10, MSE=4.145e-01
  [R=2.00] Q-train step 20, MSE=3.775e-01
  [R=2.00] Q-train step 30, MSE=3.431e-01
  [R=2.00] Q-train step 40, MSE=3.212e-01
  [R=2.00] Q-train step 50, MSE=3.086e-01
  [R=2.00] Q-train step 60, MSE=3.043e-01
2.00 |     471 | 3.040e-01 |   0.636 |      471 |  4.557e-02 |    0.851
  [R=2.05] Q-train step 10, MSE=4.265e-01
  [R=2.05] Q-train step 20, MSE=3.784e-01
  [R=2.05] Q-train step 30, MSE=3.464e-01
  [R=2.05] Q-train step 40, MSE=3.300e-01
  [R=2.05] Q-train step 50, MSE=3.181e-01
  [R=2.05] Q-train step 60, MSE=3.127e-01
2.05 |     477 | 3.125e-01 |   0.628 |      477 |  4.507e-02 |    0.852
  [R=2.10] Q-train step 10, MSE=4.429e-01
  [R=2.10] Q-train step 20, MSE=4.154e-01
  [R=2.10] Q-train step 30, MSE=3.638e-01
  [R=2.10] Q-train step 40, MSE=3.549e-01
  [R=2.10] Q-train step 50, MSE=3.383e-01
  [R=2.10] Q-train step 60, MSE=3.272e-01
2.10 |     482 | 3.263e-01 |   0.617 |      482 |  3.552e-02 |    0.875
  [R=2.15] Q-train step 10, MSE=4.579e-01
  [R=2.15] Q-train step 20, MSE=4.232e-01
  [R=2.15] Q-train step 30, MSE=3.886e-01
  [R=2.15] Q-train step 40, MSE=3.493e-01
  [R=2.15] Q-train step 50, MSE=3.370e-01
  [R=2.15] Q-train step 60, MSE=3.268e-01
2.15 |     493 | 3.260e-01 |   0.621 |      493 |  4.171e-02 |    0.860
  [R=2.20] Q-train step 10, MSE=4.569e-01
  [R=2.20] Q-train step 20, MSE=4.249e-01
  [R=2.20] Q-train step 30, MSE=3.998e-01
  [R=2.20] Q-train step 40, MSE=3.592e-01
  [R=2.20] Q-train step 50, MSE=3.428e-01
  [R=2.20] Q-train step 60, MSE=3.305e-01
2.20 |     494 | 3.297e-01 |   0.617 |      494 |  4.149e-02 |    0.857
  [R=2.25] Q-train step 10, MSE=4.575e-01
  [R=2.25] Q-train step 20, MSE=4.084e-01
  [R=2.25] Q-train step 30, MSE=3.681e-01
  [R=2.25] Q-train step 40, MSE=3.453e-01
  [R=2.25] Q-train step 50, MSE=3.327e-01
  [R=2.25] Q-train step 60, MSE=3.250e-01
2.25 |     497 | 3.247e-01 |   0.624 |      497 |  4.986e-02 |    0.844
  [R=2.30] Q-train step 10, MSE=4.615e-01
  [R=2.30] Q-train step 20, MSE=4.380e-01
  [R=2.30] Q-train step 30, MSE=4.083e-01
  [R=2.30] Q-train step 40, MSE=3.845e-01
  [R=2.30] Q-train step 50, MSE=3.729e-01
  [R=2.30] Q-train step 60, MSE=3.666e-01
2.30 |     502 | 3.658e-01 |   0.577 |      502 |  5.812e-02 |    0.799
  [R=2.35] Q-train step 10, MSE=4.681e-01
  [R=2.35] Q-train step 20, MSE=4.382e-01
  [R=2.35] Q-train step 30, MSE=4.123e-01
  [R=2.35] Q-train step 40, MSE=3.863e-01
  [R=2.35] Q-train step 50, MSE=3.751e-01
  [R=2.35] Q-train step 60, MSE=3.673e-01
2.35 |     505 | 3.660e-01 |   0.580 |      505 |  5.922e-02 |    0.798
  [R=2.40] Q-train step 10, MSE=4.676e-01
  [R=2.40] Q-train step 20, MSE=4.411e-01
  [R=2.40] Q-train step 30, MSE=4.070e-01
  [R=2.40] Q-train step 40, MSE=3.850e-01
  [R=2.40] Q-train step 50, MSE=3.783e-01
  [R=2.40] Q-train step 60, MSE=3.721e-01
2.40 |     507 | 3.712e-01 |   0.576 |      507 |  6.374e-02 |    0.783
  [R=2.45] Q-train step 10, MSE=4.778e-01
  [R=2.45] Q-train step 20, MSE=4.395e-01
  [R=2.45] Q-train step 30, MSE=3.875e-01
  [R=2.45] Q-train step 40, MSE=3.816e-01
  [R=2.45] Q-train step 50, MSE=3.695e-01
  [R=2.45] Q-train step 60, MSE=3.608e-01
2.45 |     510 | 3.599e-01 |   0.590 |      510 |  4.172e-02 |    0.850
  [R=2.50] Q-train step 10, MSE=4.726e-01
  [R=2.50] Q-train step 20, MSE=4.272e-01
  [R=2.50] Q-train step 30, MSE=3.879e-01
  [R=2.50] Q-train step 40, MSE=3.652e-01
  [R=2.50] Q-train step 50, MSE=3.535e-01
  [R=2.50] Q-train step 60, MSE=3.465e-01
2.50 |     513 | 3.460e-01 |   0.605 |      513 |  5.147e-02 |    0.829
  [R=2.55] Q-train step 10, MSE=4.797e-01
  [R=2.55] Q-train step 20, MSE=4.104e-01
  [R=2.55] Q-train step 30, MSE=3.923e-01
  [R=2.55] Q-train step 40, MSE=3.755e-01
  [R=2.55] Q-train step 50, MSE=3.670e-01
  [R=2.55] Q-train step 60, MSE=3.587e-01
2.55 |     517 | 3.577e-01 |   0.595 |      517 |  4.433e-02 |    0.843
  [R=2.60] Q-train step 10, MSE=5.033e-01
  [R=2.60] Q-train step 20, MSE=4.523e-01
  [R=2.60] Q-train step 30, MSE=4.178e-01
  [R=2.60] Q-train step 40, MSE=3.947e-01
  [R=2.60] Q-train step 50, MSE=3.858e-01
  [R=2.60] Q-train step 60, MSE=3.737e-01
2.60 |     524 | 3.725e-01 |   0.581 |      524 |  4.505e-02 |    0.843
  [R=2.65] Q-train step 10, MSE=5.082e-01
  [R=2.65] Q-train step 20, MSE=4.376e-01
  [R=2.65] Q-train step 30, MSE=4.113e-01
  [R=2.65] Q-train step 40, MSE=3.913e-01
  [R=2.65] Q-train step 50, MSE=3.810e-01
  [R=2.65] Q-train step 60, MSE=3.686e-01
2.65 |     526 | 3.677e-01 |   0.587 |      526 |  4.689e-02 |    0.838
  [R=2.70] Q-train step 10, MSE=5.018e-01
  [R=2.70] Q-train step 20, MSE=4.499e-01
  [R=2.70] Q-train step 30, MSE=4.136e-01
  [R=2.70] Q-train step 40, MSE=3.976e-01
  [R=2.70] Q-train step 50, MSE=3.880e-01
  [R=2.70] Q-train step 60, MSE=3.731e-01
2.70 |     528 | 3.718e-01 |   0.584 |      528 |  4.588e-02 |    0.842
  [R=2.75] Q-train step 10, MSE=5.238e-01
  [R=2.75] Q-train step 20, MSE=4.773e-01
  [R=2.75] Q-train step 30, MSE=4.351e-01
  [R=2.75] Q-train step 40, MSE=4.140e-01
  [R=2.75] Q-train step 50, MSE=4.053e-01
  [R=2.75] Q-train step 60, MSE=4.053e-01
2.75 |     531 | 4.050e-01 |   0.549 |      531 |  7.563e-02 |    0.747
  [R=2.80] Q-train step 10, MSE=5.232e-01
  [R=2.80] Q-train step 20, MSE=4.805e-01
  [R=2.80] Q-train step 30, MSE=4.393e-01
  [R=2.80] Q-train step 40, MSE=4.146e-01
  [R=2.80] Q-train step 50, MSE=4.054e-01
  [R=2.80] Q-train step 60, MSE=4.053e-01
2.80 |     532 | 4.050e-01 |   0.550 |      532 |  7.606e-02 |    0.745
  [R=2.85] Q-train step 10, MSE=5.287e-01
  [R=2.85] Q-train step 20, MSE=4.842e-01
  [R=2.85] Q-train step 30, MSE=4.332e-01
  [R=2.85] Q-train step 40, MSE=4.144e-01
  [R=2.85] Q-train step 50, MSE=4.106e-01
  [R=2.85] Q-train step 60, MSE=4.085e-01
2.85 |     536 | 4.080e-01 |   0.546 |      536 |  7.036e-02 |    0.759
  [R=2.90] Q-train step 10, MSE=5.227e-01
  [R=2.90] Q-train step 20, MSE=4.543e-01
  [R=2.90] Q-train step 30, MSE=4.263e-01
  [R=2.90] Q-train step 40, MSE=4.149e-01
  [R=2.90] Q-train step 50, MSE=4.036e-01
  [R=2.90] Q-train step 60, MSE=3.911e-01
2.90 |     537 | 3.894e-01 |   0.568 |      537 |  4.973e-02 |    0.827
  [R=2.95] Q-train step 10, MSE=5.242e-01
  [R=2.95] Q-train step 20, MSE=4.761e-01
  [R=2.95] Q-train step 30, MSE=4.347e-01
  [R=2.95] Q-train step 40, MSE=4.177e-01
  [R=2.95] Q-train step 50, MSE=4.088e-01
  [R=2.95] Q-train step 60, MSE=4.088e-01
2.95 |     539 | 4.084e-01 |   0.547 |      539 |  7.557e-02 |    0.746
  [R=3.00] Q-train step 10, MSE=5.219e-01
  [R=3.00] Q-train step 20, MSE=4.748e-01
  [R=3.00] Q-train step 30, MSE=4.429e-01
  [R=3.00] Q-train step 40, MSE=4.294e-01
  [R=3.00] Q-train step 50, MSE=4.121e-01
  [R=3.00] Q-train step 60, MSE=4.115e-01
3.00 |     540 | 4.109e-01 |   0.544 |      540 |  7.553e-02 |    0.749
  [R=3.05] Q-train step 10, MSE=5.473e-01
  [R=3.05] Q-train step 20, MSE=4.665e-01
  [R=3.05] Q-train step 30, MSE=4.235e-01
  [R=3.05] Q-train step 40, MSE=4.123e-01
  [R=3.05] Q-train step 50, MSE=4.058e-01
  [R=3.05] Q-train step 60, MSE=3.917e-01
3.05 |     542 | 3.903e-01 |   0.567 |      542 |  4.878e-02 |    0.829
  [R=3.10] Q-train step 10, MSE=5.460e-01
  [R=3.10] Q-train step 20, MSE=4.652e-01
  [R=3.10] Q-train step 30, MSE=4.228e-01
  [R=3.10] Q-train step 40, MSE=4.120e-01
  [R=3.10] Q-train step 50, MSE=4.045e-01
  [R=3.10] Q-train step 60, MSE=3.898e-01
3.10 |     542 | 3.884e-01 |   0.569 |      542 |  4.835e-02 |    0.831
  [R=3.15] Q-train step 10, MSE=5.410e-01
  [R=3.15] Q-train step 20, MSE=4.628e-01
  [R=3.15] Q-train step 30, MSE=4.223e-01
  [R=3.15] Q-train step 40, MSE=4.116e-01
  [R=3.15] Q-train step 50, MSE=4.027e-01
  [R=3.15] Q-train step 60, MSE=3.872e-01
3.15 |     542 | 3.857e-01 |   0.572 |      542 |  4.818e-02 |    0.833
  [R=3.20] Q-train step 10, MSE=5.218e-01
  [R=3.20] Q-train step 20, MSE=4.653e-01
  [R=3.20] Q-train step 30, MSE=4.271e-01
  [R=3.20] Q-train step 40, MSE=4.186e-01
  [R=3.20] Q-train step 50, MSE=4.061e-01
  [R=3.20] Q-train step 60, MSE=3.973e-01
3.20 |     545 | 3.962e-01 |   0.563 |      545 |  5.279e-02 |    0.816
  [R=3.25] Q-train step 10, MSE=5.672e-01
  [R=3.25] Q-train step 20, MSE=5.120e-01
  [R=3.25] Q-train step 30, MSE=4.413e-01
  [R=3.25] Q-train step 40, MSE=4.195e-01
  [R=3.25] Q-train step 50, MSE=4.095e-01
  [R=3.25] Q-train step 60, MSE=3.973e-01
3.25 |     547 | 3.967e-01 |   0.564 |      547 |  5.191e-02 |    0.817
  [R=3.30] Q-train step 10, MSE=5.295e-01
  [R=3.30] Q-train step 20, MSE=4.479e-01
  [R=3.30] Q-train step 30, MSE=4.322e-01
  [R=3.30] Q-train step 40, MSE=4.121e-01
  [R=3.30] Q-train step 50, MSE=4.047e-01
  [R=3.30] Q-train step 60, MSE=3.938e-01
3.30 |     547 | 3.928e-01 |   0.568 |      547 |  4.992e-02 |    0.825
  [R=3.35] Q-train step 10, MSE=5.306e-01
  [R=3.35] Q-train step 20, MSE=4.621e-01
  [R=3.35] Q-train step 30, MSE=4.372e-01
  [R=3.35] Q-train step 40, MSE=4.188e-01
  [R=3.35] Q-train step 50, MSE=4.075e-01
  [R=3.35] Q-train step 60, MSE=3.952e-01
3.35 |     550 | 3.944e-01 |   0.567 |      550 |  5.181e-02 |    0.818
  [R=3.40] Q-train step 10, MSE=5.833e-01
  [R=3.40] Q-train step 20, MSE=5.199e-01
  [R=3.40] Q-train step 30, MSE=4.628e-01
  [R=3.40] Q-train step 40, MSE=4.299e-01
  [R=3.40] Q-train step 50, MSE=4.280e-01
  [R=3.40] Q-train step 60, MSE=4.248e-01
3.40 |     550 | 4.248e-01 |   0.534 |      550 |  8.173e-02 |    0.728
  [R=3.45] Q-train step 10, MSE=5.348e-01
  [R=3.45] Q-train step 20, MSE=4.731e-01
  [R=3.45] Q-train step 30, MSE=4.358e-01
  [R=3.45] Q-train step 40, MSE=4.220e-01
  [R=3.45] Q-train step 50, MSE=4.080e-01
  [R=3.45] Q-train step 60, MSE=3.960e-01
3.45 |     550 | 3.953e-01 |   0.566 |      550 |  5.138e-02 |    0.819
...
"""  # <- truncated for brevity in this snippet; paste the entire console dump here
# ------------------------------------------------------------------
# 2.  Parse every table row: "<R> | <calls_q> | <mse_q> | <r2_q> | <calls_ad> | <mse_ad> | <r2_ad>"
# ------------------------------------------------------------------
pattern = re.compile(
    r"""^\s*
        (?P<R>\d+\.\d+)\s*\|\s*
        (?P<calls_q>\d+)\s*\|\s*
        (?P<mse_q>\d+\.\d+e[+-]?\d+)\s*\|\s*
        (?P<r2_q>[-+]?\d+\.\d+)\s*\|\s*
        (?P<calls_ad>\d+)\s*\|\s*
        (?P<mse_ad>\d+\.\d+e[+-]?\d+)\s*\|\s*
        (?P<r2_ad>[-+]?\d+\.\d+)
     """,
    re.VERBOSE | re.MULTILINE
)

rows = [m.groupdict() for m in pattern.finditer(raw_data)]
if not rows:
    raise ValueError("No valid rows were parsed — check that raw_data contains the full console output.")

df = pd.DataFrame(rows).astype({
    "R": float,
    "calls_q": int,
    "mse_q": float,
    "r2_q": float,
    "calls_ad": int,
    "mse_ad": float,
    "r2_ad": float,
})

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ------------------------------------------------------------------
# 3.  Three plots in a single row
# ------------------------------------------------------------------
# Create subplots with 1 row and 3 columns
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Quantum surrogate: R² vs. patch radius",
                   "Classical separable surrogate: R² vs. patch radius",
                   "Quantum calls vs. patch radius"),
    horizontal_spacing=0.12  # Adjust spacing between plots
)

# Add the three line plots to the subplots
fig.add_trace(
    go.Scatter(x=df["R"], y=df["r2_q"], mode='lines+markers', name="Quantum R²"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df["R"], y=df["r2_ad"], mode='lines+markers', name="Classical R²"),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=df["R"], y=df["calls_q"], mode='lines+markers', name="Quantum Calls"),
    row=1, col=3
)

# Update axis labels
fig.update_xaxes(title_text="Patch Radius", row=1, col=1)
fig.update_xaxes(title_text="Patch Radius", row=1, col=2)
fig.update_xaxes(title_text="Patch Radius", row=1, col=3)

fig.update_yaxes(title_text="Local R² (Quantum)", row=1, col=1)
fig.update_yaxes(title_text="Local R² (Classical)", row=1, col=2)
fig.update_yaxes(title_text="Quantum Calls", row=1, col=3)

# Update layout
fig.update_layout(
    height=400,  # Adjust height as needed
    width=1200,  # Adjust width as needed
    showlegend=False  # Hide legend since titles are descriptive
)

fig.show()